In [ ]:
import MDAnalysis as mda
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pp
import analyse as ana
from scipy.stats import gaussian_kde as kde
from IPython.display import display, Markdown, Latex
from importlib import reload
%matplotlib widget

In [ ]:
from matplotlib.font_manager import FontProperties

textwidth = 6.50128 # latex textwidth

font0 = FontProperties()

font = font0.copy()
font.set_family('sans-serif')
font.set_style('normal')
font.set_weight('bold')
font.set_size(16)

Title = font.copy()
Title.set_size(18)

font2 = Title.copy()
font2.set_size(12)

font3 = font.copy()
font3.set_size(12)
font3.set_weight('normal')

mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['axes.titlesize'] = 18
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['font.size'] = 12
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['axes.labelweight'] = 'bold'
mpl.rcParams['font.family'] = 'sans-serif'

mpl.rcParams['figure.figsize'] = (textwidth, 4)

## Functions, definitions, and universe creations.

In [ ]:
def plot(axes, unis, inputs, uni_func, parse_func):
    for n, inp in enumerate(inputs):
        ax = axes.flatten()[n]
        ax.set_title(inp['label'])

        for par, uni in unis.items():
            #display(Markdown(f'## {par}'))
            sels = parse_func(inp, par)
            data = getattr(uni, uni_func)(*sels)
            _, bins = np.histogram(data)
            #print(data)
            
            data_range = np.linspace(bins[0], bins[-1], 200)
            
            kwargs = dict(
                label=par,
                linewidth=2
            )
            if par == 'NMR': 
                kwargs['color'] = 'k'
                kwargs['linestyle'] = '--'

                
            lines = ax.plot(
                data_range, kde(data)(data_range), 
                **kwargs
            )

            mean = np.mean(data)
            kwargs['color'] = lines[-1].get_color()
            kwargs['label'] = None
            ax.axvline(mean, **kwargs)
            

### Parameter sets

parsets = [
   'CTPOL', 'opt-CTPOL'
]

ref = 'trajectories/input.pdb'

### Inputs for angle calculating function

### Get Universes

reload(ana)

unis = dict(NMR=ana.Universe(ref, ref))
for par in parsets:
    dcd = f"trajectories/MD-1ZNF/{par}/centered_output.dcd"
    unis[par] = ana.Universe('output.pdb', dcd)

### Plot

## Distributions of various angles

In [ ]:
inputs = [
    dict(
        label = "S4-Zn-S7",
        selections = [
            dict(name='SG', resid=4),
            dict(name='ZN'),
            dict(name='SG', resid=7)
        ]
    ),
    dict(
        label = "N20-Zn-N24",
        selections = [
            dict(name='NE2', resid=20),
            dict(name='ZN'),
            dict(name='NE2', resid=24)
        ]
    ),
    dict(
        label = "S4-Zn-N20",
        selections = [
            dict(name='SG', resid=4),
            dict(name='ZN'),
            dict(name='NE2', resid=20)
        ]
    ),
    dict(
        label = "S4-Zn-N24",
        selections = [
            dict(name='SG', resid=4),
            dict(name='ZN'),
            dict(name='NE2', resid=24)
        ]
        )
]

def parse_angles(inp, par):
    sels = []
    for s in inp['selections']:
        sel = s.copy()
        if 'resid' in sel:
            sel['resid'] -= (par == 'NMR')*1
            
        seltxt = ' and '.join(
            [f'{k} {v}' for k, v in sel.items()]
        )
        sels.append(unis[par].select_atoms(seltxt))
    return sels

In [ ]:
fig, axes = pp.subplots(2,2, sharex=True, sharey=True)
fig.subplots_adjust(
    wspace=0.05,
    hspace=0.25,
    top = 0.90,
    left = 0.1,
    right = 0.975
)
#fig.suptitle("Probability distribution of angles (in degrees)")   


#fig.set_size_inches(7,5)



plot(axes, unis, inputs, 'get_angles', parse_angles)

In [ ]:
pp.gca().set_xlim(85,135)
pp.gca().set_ylim(0, None)

In [ ]:
leg = pp.gca().legend()
pp.gca().set_xlim(80,144)
pp.gca().set_ylim(0, None)
fig

In [ ]:
fig.savefig('Images/Angular_distributions.pdf', dpi=300)

## A view of the protein

In [ ]:
uni = unis['CTPOL']
protein = uni.select_atoms('protein')
site = uni.select_atoms('same residue as (around 3 name ZN)')
zn = uni.select_atoms('name ZN')
view = nv.show_mdanalysis(protein)
view

In [ ]:
view.add_trajectory(site)
view[-1].add_representation('ball+stick')
view.add_trajectory(zn)
view[-1].add_representation('ball+stick')

## Distributions of dihedrals and other angles between planes

### Angles between the two Histidine planes

In [ ]:
inputs = [
    dict(
        label = "Angle between His1 and His2 planes",
        selections = [
            dict(atoms='(name CG or name CD2 or name CE1)', resid=20),
            dict(atoms='(name CG or name CD2 or name CE1)', resid=24)
        ]
    ),
    dict(
        label = "Angle between His1 C and His1 N planes",
        selections = [
            dict(atoms='(name CG or name CD2 or name CE1)', resid=20),
            dict(atoms='(name CG or name ND1 or name NE2)', resid=20)
        ]
    ), 
]

def parse_his(inp, par):
    sels = []
    for s in inp['selections']:
        sel = s.copy()
        if 'resid' in sel:
            sel['resid'] -= (par == 'NMR')*1
            
        seltxt = f'{sel["atoms"]} and resid {sel["resid"]}'
        sels.append(unis[par].select_atoms(seltxt))
    return sels    

fig, axes = pp.subplots(2,1)
fig.subplots_adjust(hspace=0.4)
plot(axes, unis, inputs, 'get_dihedral', parse_his)

In [ ]:
axes[0].set_xlim(20, 150)
axes[1].set_xlim(165, 180)
for ax in axes:
    ax.set_ylim(0, None)
fig

In [ ]:
fig.savefig('Images/ang_between_his.pdf', dpi=300)

### Dihedral between S-ZN-S and N-Zn-N planes

In [ ]:
def parse_binding(inp, par):
    sels = []
    for plane in inp['selections']:
        atoms = []
        for s in plane:
            sel = s.copy()
            if 'resid' in sel:
                sel['resid'] -= (par == 'NMR')*1
            
            seltxt = ' and '.join(
                [f'{k} {v}' for k, v in sel.items()]
            )
            
            atoms.append(unis[par].select_atoms(seltxt))
        sels.append(sum(atoms))
    return sels 

inputs = [
    dict(
        label = "",#"Angle between N-Zn-N and S-Zn-S planes",
        selections = [
            [
                dict(name='SG', resid=4),
                dict(name='ZN'),
                dict(name='SG', resid=7)
            ],
            [
                dict(name='NE2', resid=20),
                dict(name='ZN'),
                dict(name='NE2', resid=24)
            ]
        ]
    )
]


In [ ]:
fig, ax = pp.subplots()
fig.set_size_inches(textwidth/2, 2)
fig.subplots_adjust(
    left=0.2,
    right=0.95
    
)
plot(np.array([ax]), unis, inputs, 'get_dihedral', parse_binding)

In [ ]:

ax.set_xlim(77,110)
ax.set_ylim(0, 0.125)
fig

In [ ]:
leg = ax.legend()

In [ ]:
fig.savefig('Images/dihedral_between_SZnS_NZnN.pdf', dpi=300)

### Angle between bisectors of S-ZN-S and N-ZN-N

In [ ]:
inputs = [
    dict(
        label = "",#"Angle between N-Zn-N and S-Zn-S bisectors",
        selections = [
            [
                dict(name='SG', resid=4),
                dict(name='ZN'),
                dict(name='SG', resid=7)
            ],
            [
                dict(name='NE2', resid=20),
                dict(name='ZN'),
                dict(name='NE2', resid=24)
            ]
        ]
    )
]

In [ ]:
fig, ax = pp.subplots()
fig.set_size_inches(textwidth/2, 2)
fig.subplots_adjust(
    left=0.15,
    right=0.95
    
)
plot(np.array([ax]), unis, inputs, 'get_bisector_angles', parse_binding)

In [ ]:
ax.set_xlim(120, 180)
ax.set_ylim(0, 0.15)
fig

In [ ]:
leg = ax.legend()

In [ ]:
fig.savefig('Images/bisector_angles.pdf', dpi=300)

# Todos
* See how we calculate chargs transfer (plot $\Delta$q vs $r$)
* Rerun with more frequent update of CT.
* Describe polarization